# 2698. Find the Punishment Number of an Integer

[2698. Find the Punishment Number of an Integer](https://leetcode.com/problems/find-the-punishment-number-of-an-integer/description/) is a leetcode problem with one of the most threatening names I've ever seen. The malicious aura here is palpable.

I also hate it. It has one of the magic tricks that I often malign. But let's take a look at the problem before I get more editorial.

> Given a positive integer `n`, return the punishment number of `n`.
> The punishment number of `n` is defined as the sum of the squares of all integers `i` such that:
>    - 1 <= i <= n
>    - The decimal representation of `i * i` can be partitioned into contiguous substrings such that the sum of the integer values of these substrings equals i.

> Example 1:
> Input: `n = 10`
> Output: 182
> Explanation: There are exactly 3 integers `i` in the range [1, 10] that satisfy the conditions in the statement:
> - 1 since 1 * 1 = 1
> - 9 since 9 * 9 = 81 and 81 can be partitioned into 8 and 1 with a sum equal to 8 + 1 == 9.
> - 10 since 10 * 10 = 100 and 100 can be partitioned into 10 and 0 with a sum equal to 10 + 0 == 10.
> Hence, the punishment number of 10 is 1 + 81 + 100 = 182

> Example 2:
> Input: `n = 37`
> Output: 1478
> Explanation: There are exactly 4 integers `i` in the range [1, 37] that satisfy the conditions in the statement:
> - 1 since 1 * 1 = 1. 
> - 9 since 9 * 9 = 81 and 81 can be partitioned into 8 + 1. 
> - 10 since 10 * 10 = 100 and 100 can be partitioned into 10 + 0. 
> - 36 since 36 * 36 = 1296 and 1296 can be partitioned into 1 + 29 + 6.
> Hence, the punishment number of 37 is 1 + 81 + 100 + 1296 = 1478
>
>Constraints:
>    1 <= n <= 1000

From here it should be clear why I hate this problem. That's right: whoever wrote the examples used the word "Hence" like they're better than you.

But also because I maintain that any coding question that could have an OEIS page is a bad interview problem. Like [Fibonacci Numbers](https://oeis.org/A000045). If you got asked to do Fibonacci Numbers in an interview, you'd laugh, right?

But at any rate, this is a 2 part problem:
1. Find all the numbers `i` between 1 and `n` that qualify.
2. Square all qualified numbers and return the sum. (e.g. 1+81+100 = 182)

Let's also note that Example 2 was chosen very deliberately. It's *almost* the next number after 10 where the punishment number is any bigger, which would have been 36. That's right, all pn(n) where 10 < n < 36, pn(n) = 182.

As for the constraints, it seems to be mostly because square numbers can get away from you. 10,000<sup>2</sup> would add another 2 zeroes and 100,000<sup>2</sup> would no longer fit in a 32 bit integer.

So, to codify what we have:


In [1]:
def punishment_numbers(n):
    def can_partition(num, i):
        # if i * i can be partitioned 
        # into contiguous substrings 
        # such that the sum of the integer
        # values of these substrings equals i.
        return False
    
    def qualify_numbers(n):
        for i in range(n):
            if can_partition(str(i**2), i):
                yield i

    def square_and_sum(qualified):
        return sum(i ** 2 for i in qualified)

    return square_and_sum(qualify_numbers(n))

Okay so everything except the `can_partition` part is basically done. We just need to implement that. How hard could it be?

That's only half-foreshadowing for later.

Let's re-examine those examples. Just because we read them doesn't mean we understand their implications. Specifically, we're going to look at the numbers we were told qualify and their squares: 1:1, 9:81, 10:100 and 36:1296.

With your beautiful human eyes it's trivially easy to see things like `1==1` and `10+0 == 10`, but we need to be able to tell a computer how to do that.

So let's pretend for a second we lack those human eyes and go back to AP Algebra.

- Given x-1=0, solve for x.
- `x+1-1=0+1` Add x to both sides...
- `x=1` Collapse terms and oh dang x was 1.

We'll need to do something similar with our numbers to see if they qualify. Let's take 36 for example:

- `36 * 36 = 1296` Initialize
- `36 = '1296'` Cast to string, drop the extra term.
We know from the example the valid partition here is 1|29|6.
- `36 - 1 = '1296'[1:]` "take" 1 from both sides.
- `35 = '296'` Settle terms
- `35 - 6 = '296'[:2]` Take the 6 off this time
- `29 = 29` Settle terms.
- `29 - 29 = '29'[:2]` Take the entire string this time.
- `0 = ''`. True. 36 qualifies.

**Figuring out how to write this in code is dumb magic. Either you see it or you don't**.



So while we write our function, we'll need to:
- Take a str repr of a number
- Keep track of a moving target (36->35->29->0)
- Take from either the left or the right some amount of digits
- Return `True` if there is a path that finds `0=''` or that our target number is the same as our search number.
- Return `False` if we did not find such a path.

Let's take another look at that example.

> - `36 - 1 = '1296'[1:]` "take" 1 from both sides.
> - `35 = '296'` Settle terms
> - `35 - 6 = '296'[:2]` Take the 6 off this time
> - `29 = 29` Settle terms.

What if we did it with variables?

```
i = 36, i ** 2 = 1296
target = 36, str = '1296'
target = target - 6, str = str[:-1]

target = 30, str = '129'
target = target - 1, str = str[1:]

target = 29, str = '29'
target = 29-29, str = ''
```

Hey wait a second that's secretly a recurrence relation. See all the duplicated work?

That means we need to write a recursive function. Hooray!

Let's also consider some INVALID paths.
```
1. target = 36, str = '1296'
2. target = target - 1, str = str[1:]
3. target = 36, str = '296'
4. target = target - 2, str = str[1:]
5. target = 36, str = '96'
6. target = target - 9, str = str[1:]
7. target = 36, str = '6'
...
```
But if at step 4, we had chosen 2 to slice with instead, we would have a valid path.

So we need to increment over our string and attempt each index.

Finally, imagine if we subtracted 123 and had '6' left in our string. 36-123 is obviously way below 36 right? We missed our mark.

If our target ever becomes negative, we overshot and can safely say it's *not* a qualifying number.

Okay Code Time<sup>TM</sup>.

In [2]:
from functools import cache

def punishment_numbers(n):

    @cache
    def can_partition(str_num, target):
        if str_num=='' and target==0:
            return True
        if target < 0:
            return False 

        for i in range(len(str_num)):
            l_part = str_num[:i+1]
            r_part = str_num[i+1:]
            assert l_part+r_part == str_num
            target_adjustment = int(l_part)
            if can_partition(r_part, target-target_adjustment):
                return True
        
        return False
    
    def qualify_numbers(n):
        for i in range(1, n+1):
            if can_partition(str(i**2), i):
                yield i

    def square_and_sum(qualified):
        return sum(i ** 2 for i in qualified)

    return square_and_sum(qualify_numbers(n))

punishment_numbers(10)

182

In [3]:
%%timeit
punishment_numbers(10)

18.4 μs ± 368 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


We did it, Patrick! We saved the city.

Well, actually, there's another bit of magic there that I'm just kinda glossing over.

This part:
```python
        for i in range(0, len(str_num)):
            l_part = str_num[:i+1]
            r_part = str_num[i+1:]
            assert l_part+r_part == str_num
            # Why chose the left side as the adjustment?
            target_adjustment = int(l_part)
            # Why always check if we can partition the remaining right side?
            if can_partition(r_part, target-target_adjustment): 
                return True
```

Essentially the code as written will always have a valid l_part. As we iterate over 100, we try against 1, 10 and 100. What's left on the right is 00, 0 and an empty string.

However, there's more Magic✨ we can do with this problem.

Did you know by repeatedly summing the digits of a number, you can find out if it's divisible by nine? [Effed up but True](https://en.wikipedia.org/wiki/Casting_out_nines). It turns out that any number this works for will have the same digital root as its square, which sounds like something I just made up but I promise I didn't.

In [4]:
def cast_out_nines(num):
    n = str(num)
    cast = sum(int(i) for i in n)
    return cast

def repeat_nines(num):
    n = cast_out_nines(num)
    while n > 9:
        n = cast_out_nines(n)
    return n
    
for i in range(1, 40):
    digital_root = repeat_nines(i)
    if digital_root == repeat_nines(i** 2):
        print(f"\t{i}\t{i**2}\tshare digital root {digital_root}")

	1	1	share digital root 1
	9	81	share digital root 9
	10	100	share digital root 1
	18	324	share digital root 9
	19	361	share digital root 1
	27	729	share digital root 9
	28	784	share digital root 1
	36	1296	share digital root 9
	37	1369	share digital root 1


Yup there's 1, 9, 10 and 36.

But there's also a new pattern. All of those numbers, which we know to be Punishment Qualified, have a digital root of either 1 or 9.

But again that's why I consider this Magic Bullshit✨. If you know that, this question is easy. If you don't know that you read a Jupyter Notebook out of some dudes GitHub repo desparately trying to understand the problem before your interview at Amazon tomorrow morning.

Moving on, that means we can simplify our search and cut the problem space down like 80%.

In [5]:
from functools import cache, partial
 # <3 u partial
mod_filter = partial(filter, lambda x: x % 9 == 0 or x % 9 ==1)

def punishment_numbers(n):

    @cache
    def can_partition(str_num, target):
        if str_num=='' and target==0:
            return True
        if target < 0:
            return False 

        for i in range(len(str_num)):
            l_part = str_num[:i+1]
            r_part = str_num[i+1:]
            target_adjustment = int(l_part)
            if can_partition(r_part, target-target_adjustment):
                return True
        
        return False
    
    def qualify_numbers(n):
        for i in mod_filter(range(1, n+1)):
            if can_partition(str(i**2), i):
                yield i

    def square_and_sum(qualified):
        return sum(i ** 2 for i in qualified)

    return square_and_sum(qualify_numbers(n))

punishment_numbers(10)

182

In [6]:
%%timeit
punishment_numbers(10)

8.89 μs ± 53.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Now at this point you're probably like
> Robert, this is great. Surely the last step to getting the FASTEST solution on Leetcode, is to convert our recursive can_partition function into an iterative one?

And while that's very astute of you, nah I'm just gonna cheat.
> Pardon?

No, you heard (read?) me.

You see, there's a very specific reason that I've written the example code the way I have, and it's not because that's how I submitted the code the first time. Like I mentioned at the start of the file, this is a two part problem, but we've been combining them.

So what if, for the sake of argument, we changed a line of code? Just one.

In [7]:
from functools import cache, partial
 # <3 u partial
mod_filter = partial(filter, lambda x: x % 9 == 0 or x % 9 ==1)

def punishment_numbers(n):

    @cache
    def can_partition(str_num, target):
        if str_num=='' and target==0:
            return True
        if target < 0:
            return False 

        for i in range(len(str_num)):
            l_part = str_num[:i+1]
            r_part = str_num[i+1:]
            target_adjustment = int(l_part)
            if can_partition(r_part, target-target_adjustment):
                return True
        
        return False
    
    def qualify_numbers(n):
        for i in mod_filter(range(1, n+1)):
            if can_partition(str(i**2), i):
                yield i

    def square_and_sum(qualified):
        return sum(i ** 2 for i in qualified)

    return list(qualify_numbers(n))  # This one.

punishment_numbers(10)

[1, 9, 10]

If you remember from our constraints: n <= 1000.

So what if we just... do them all? What if those numbers were just known? Like, say, *most OEIS sequences*?

In [8]:
qualifying = punishment_numbers(1000)
print(f"{qualifying[:5]}...{qualifying[-5:]}")

[1, 9, 10, 36, 45]...[964, 990, 991, 999, 1000]


Because if we do that, our function just becomes...

In [9]:
from functools import cache, partial
 # <3 u partial
mod_filter = partial(filter, lambda x: x % 9 == 0 or x % 9 ==1)

def punishment_numbers(n):

    # @cache
    # def can_partition(str_num, target):
    #     if str_num=='' and target==0:
    #         return True
    #     if target < 0:
    #         return False 

    #     for i in range(len(str_num)):
    #         l_part = str_num[:i+1]
    #         r_part = str_num[i+1:]
    #         target_adjustment = int(l_part)
    #         if can_partition(r_part, target-target_adjustment):
    #             return True
        
    #     return False
    
    # def qualify_numbers(n):
    #     for i in mod_filter(range(1, n+1)):
    #         if can_partition(str(i**2), i):
    #             yield i

    def square_and_sum(n):
        return sum(i ** 2 for i in qualifying if i <= n)

    # We don't even need the extra function anymore but I want it there for the sake of
    # code continuity.
    return square_and_sum(n)

punishment_numbers(10)

182

In [10]:
# and NOW we refactor.
precomputed = [
    1,9,10,36,45,55,82,91,99,100,235,297,369,370,379,
    414,657,675,703,756,792,909,918,945,964,990,991,999,1000,
]

def punishment_number(n):
    return sum(i**2 for i in precomputed if i <= n)

In [11]:
%%timeit
punishment_numbers(1000)

2.58 μs ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Ah, but can we make faster, more unreadable (and therefore better) code? Of course, done here through the power of metaprogramming.

In [12]:
def punishment_number(n):
    return sum(i**2 for i in precomputed if i <= n)

def output_punishment():
    precomputed = [
        1,9,10,36,45,55,82,91,99,100,235,297,369,370,379,
        414,657,675,703,756,792,909,918,945,964,990,991,999,1000,
    ]

    code_out = """if n == 1:\n\treturn 1\n"""
    statements = []
    for qn in precomputed[1:]:
        statements.append(f"""elif n <= {qn}:\n\treturn {punishment_number(qn)}""")
    return code_out + "\n".join(statements)

print(output_punishment())

if n == 1:
	return 1
elif n <= 9:
	return 82
elif n <= 10:
	return 182
elif n <= 36:
	return 1478
elif n <= 45:
	return 3503
elif n <= 55:
	return 6528
elif n <= 82:
	return 13252
elif n <= 91:
	return 21533
elif n <= 99:
	return 31334
elif n <= 100:
	return 41334
elif n <= 235:
	return 96559
elif n <= 297:
	return 184768
elif n <= 369:
	return 320929
elif n <= 370:
	return 457829
elif n <= 379:
	return 601470
elif n <= 414:
	return 772866
elif n <= 657:
	return 1204515
elif n <= 675:
	return 1660140
elif n <= 703:
	return 2154349
elif n <= 756:
	return 2725885
elif n <= 792:
	return 3353149
elif n <= 909:
	return 4179430
elif n <= 918:
	return 5022154
elif n <= 945:
	return 5915179
elif n <= 964:
	return 6844475
elif n <= 990:
	return 7824575
elif n <= 991:
	return 8806656
elif n <= 999:
	return 9804657
elif n <= 1000:
	return 10804657


In [13]:
def punishment_number(n):
    if n == 1:
    	return 1
    elif n <= 9:
    	return 82
    elif n <= 10:
    	return 182
    elif n <= 36:
    	return 1478
    elif n <= 45:
    	return 3503
    elif n <= 55:
    	return 6528
    elif n <= 82:
    	return 13252
    elif n <= 91:
    	return 21533
    elif n <= 99:
    	return 31334
    elif n <= 100:
    	return 41334
    elif n <= 235:
    	return 96559
    elif n <= 297:
    	return 184768
    elif n <= 369:
    	return 320929
    elif n <= 370:
    	return 457829
    elif n <= 379:
    	return 601470
    elif n <= 414:
    	return 772866
    elif n <= 657:
    	return 1204515
    elif n <= 675:
    	return 1660140
    elif n <= 703:
    	return 2154349
    elif n <= 756:
    	return 2725885
    elif n <= 792:
    	return 3353149
    elif n <= 909:
    	return 4179430
    elif n <= 918:
    	return 5022154
    elif n <= 945:
    	return 5915179
    elif n <= 964:
    	return 6844475
    elif n <= 990:
    	return 7824575
    elif n <= 991:
    	return 8806656
    elif n <= 999:
    	return 9804657
    elif n <= 1000:
    	return 10804657

In [14]:
%%timeit
punishment_number(1000)

227 ns ± 1.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


And THAT, is what you actually put on Leetcode. Completely unreadable garbage code that in any sane world would get you a taking to by HR. But it's so fast. 😇

I hate problems like this.

Happy coding!